## Object Detection

In [ ]:
import json

from os import listdir, makedirs, path

from PIL import Image as PImage

from dominant_colors import get_dominant_colors, resize_PIL
from objects_models import Owl2

from parameters.arquigrafia import IMAGES_PATH, OBJECTS_PATH, OBJECTS, OBJECT2LABEL

makedirs(OBJECTS_PATH, exist_ok=True)

In [ ]:
OBJS_LABELS_IN = [sorted(o.keys()) for o in OBJECTS]
OBJS_LABELS_OUT = [[OBJECT2LABEL.get(l, l) for l in oli] for oli in OBJS_LABELS_IN]
OBJS_THOLDS = [[OBJECTS[i][k] for k in oli] for i,oli in enumerate(OBJS_LABELS_IN)]

### Run Object Detection

In [ ]:
%%time

input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

for io_file in input_files[:4096]:
  input_file_path = path.join(IMAGES_PATH, io_file)
  output_file_path = path.join(OBJECTS_PATH, io_file.replace(".jpg", ".json"))

  if path.isfile(output_file_path):
    continue

  if int(io_file.replace(".jpg", "")) % 50 == 0:
    print(IMAGES_PATH, io_file)

  image = PImage.open(input_file_path).convert("RGB")

  rgb_by_count, rgb_by_hls = get_dominant_colors(resize_PIL(image))

  image_data = {}

  image_data["boxes"] = {}
  for i in range(len(OBJS_LABELS_IN)):
    obj_boxes = Owl2.top_objects(image, OBJS_LABELS_IN[i], OBJS_LABELS_OUT[i], OBJS_THOLDS[i])
    image_data["boxes"] = image_data["boxes"] | obj_boxes

  image_data["dominant_color"] = {
    "by_count": [int(v) for v in rgb_by_count[0]],
    "by_hue": [int(v) for v in rgb_by_hls[0]],
    "palette": [[int(v) for v in c] for c in rgb_by_hls[:4]],
  }

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_data, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Post-Process: Create output json file

In [ ]:
from parameters.arquigrafia import CAPTIONS_PATH, OBJECTS_PATH, DB_FILE_PATH
from export_utils import export_objs_caps

export_objs_caps(OBJECTS_PATH, CAPTIONS_PATH, DB_FILE_PATH)

### Post-Process: Create separate json files

In [ ]:
from parameters.arquigrafia import OBJECTS_PATH
from export_utils import export_by_keys

keys = ["binaries", "boxes"]
export_by_keys(OBJECTS_PATH, keys)

### Count objects

In [ ]:
import json

from parameters.arquigrafia import DB_FILE_PATH, LABEL2DISPLAY

with open(DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

object_count = sorted([(LABEL2DISPLAY[k], len(a)) for k, a in obj_data.items()], key=lambda x: x[1], reverse=True)

for o,c in object_count:
  print(o, ":", c)


### TEST: boxes from JSON

In [ ]:
import json

from os import path
from PIL import Image as PImage, ImageDraw as PImageDraw, ImageFont as PImageFont

from parameters.arquigrafia import IMAGES_PATH, DB_FILE_PATH, LABEL2DISPLAY

MFONT = PImageFont.load_default(20)

with open(DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

for id, d in list(img_data.items())[:3]:
  img_path = path.join(IMAGES_PATH, f"{id}.jpg")
  img = PImage.open(img_path).convert("RGBA")
  iw,ih = img.size
  draw = PImageDraw.Draw(img)
  for label, (x0,y0,x1,y1) in d["boxes"].items():
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)
  print([LABEL2DISPLAY[l] for l in d["boxes"].keys()], "\n", d["captions"]["en"]["blip"])
  display(img)

### TEST: Dominant Color

In [ ]:
import numpy as np

from os import listdir, path
from PIL import Image as PImage

from parameters.arquigrafia import IMAGES_PATH
from dominant_colors import get_dominant_colors, resize_PIL

INPUT_FILES = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

io_file = INPUT_FILES[0]
input_file_path = path.join(IMAGES_PATH, io_file)

In [ ]:
image = PImage.open(input_file_path).convert("RGB")
image_s = resize_PIL(image)
rgb_by_count, rgb_by_hls = get_dominant_colors(image_s)

In [ ]:
iw, ih = [(d // 2) * 2 for d in image_s.size]
image_shape = (ih, iw, 3)
ppc = int(ih * iw / len(rgb_by_count))

count_np_image = np.array([ppc * [c] for c in rgb_by_count]).reshape(image_shape)
hls_np_image = np.array([ppc * [c] for c in rgb_by_hls]).reshape(image_shape)

display(image_s)
display(PImage.fromarray(count_np_image))
display(PImage.fromarray(hls_np_image))

### Separate/Copy image files for threshold adjustment

In [ ]:
import shutil

from os import path, makedirs

from parameters.arquigrafia import IMAGES_PATH

tocopy = [
  'vehicle',
]

for o in tocopy:
  print(o, len(obj_data[o]), obj_data[o], "\n")
  img_out_dir = f"test-{o.replace(' ', '-')}"
  img_out_dir_path = path.join(IMAGES_PATH.replace("arquigrafia", img_out_dir))
  makedirs(img_out_dir_path, exist_ok=True)
  for i in obj_data[o]:
    img_in_path = path.join(IMAGES_PATH, f"{i}.jpg")
    shutil.copy2(img_in_path, img_out_dir_path)

### Adjust Thresholds

In [ ]:
import json

from os import path, listdir
from PIL import Image as PImage, ImageDraw as PImageDraw, ImageFont as PImageFont

from models import Owl2
from parameters.finetune import IMAGES_PATH, DB_FILE_PATH, OBJECTS, OBJECT2LABEL

with open(DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

print(obj_data.keys(), len(obj_data.keys()))

In [ ]:
MFONT = PImageFont.load_default(20)

OBJS_LABELS_IN = [sorted(o.keys()) for o in OBJECTS]
OBJS_LABELS_OUT = [[OBJECT2LABEL.get(l, l) for l in oli] for oli in OBJS_LABELS_IN]
OBJS_THOLDS = [[OBJECTS[i][k] for k in oli] for i,oli in enumerate(OBJS_LABELS_IN)]

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

for io_file in input_files:
  input_file_path = path.join(IMAGES_PATH, io_file)

  image = PImage.open(input_file_path).convert("RGB")
  iw,ih = image.size
  print(image.size)

  objs = []
  for i in range(len(OBJS_LABELS_IN)):
    objs += Owl2.all_objects(image, OBJS_LABELS_IN[i], OBJS_LABELS_OUT[i], OBJS_THOLDS[i])

  print([f'{o["label"]}: {o["score"]}' for o in objs])

  draw = PImageDraw.Draw(image)
  for o in objs:
    (x0,y0,x1,y1) = o["box"]
    score, label = o["score"].item(), o["label"]
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)
    draw.text((x0*iw, y0*ih + 20), f"{round(score, 3)}", (255, 255, 255), font=MFONT)
    draw.text((x0*iw, y0*ih - 0), f"{label}", (255, 0, 0), font=MFONT)
  display(image)